In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import os
from tqdm.notebook import tqdm
from pathlib import Path
from rdkit import Chem

# Publication-Ready Filtering Step Figures

Generate two publication-quality figures showing unique molecules remaining after each filtering step:
1. **Without overlap filtering** (Initial → Assay Size)
2. **With overlap filtering** (Initial → Max Overlap)

Using Plotly for fancier, interactive visualizations with highlighted first/last bars.

## Get Actual Counts from Filtering Notebook

To get the exact total activity counts, run this code in the `filter_chembl_data_refined.ipynb` notebook (replace `.nunique()` with `len()`):

```python
# Replace step_counts.append(current_df['molecule_chembl_id'].nunique())
# With:    step_counts.append(len(current_df))
```

Or run the cell below to compute counts from the raw data:

In [6]:
# Compute actual counts from raw data
# This replicates the filtering logic and counts both rows (activities) and unique molecules

raw_path = Path("../data/chembl_affinity_raw.parquet")
minoverlap_path = Path("../data/minoverlap_chembl_affinity.parquet")
maxoverlap_path = Path("../data/maxoverlap_chembl_affinity.parquet")

# Initialize count dictionaries
COUNTS_TOTAL_ACTIVITIES = {}
COUNTS_UNIQUE_MOLECULES = {}

if raw_path.exists():
    print("Loading raw data to compute actual counts...")
    full_df = pd.read_parquet(raw_path)
    
    # Initial
    current_df = full_df.copy()
    COUNTS_TOTAL_ACTIVITIES['Initial'] = len(current_df)
    COUNTS_UNIQUE_MOLECULES['Initial'] = current_df['InchiKey'].nunique() if 'InchiKey' in current_df.columns else current_df['canonical_smiles'].nunique()
    
    # Type filter
    current_df = current_df[current_df['standard_type'].isin(['Ki', 'IC50'])]
    COUNTS_TOTAL_ACTIVITIES['Type'] = len(current_df)
    COUNTS_UNIQUE_MOLECULES['Type'] = current_df['InchiKey'].nunique() if 'InchiKey' in current_df.columns else current_df['canonical_smiles'].nunique()
    
    # Units filter
    current_df = current_df[current_df['standard_units'] == 'nM']
    COUNTS_TOTAL_ACTIVITIES['Units'] = len(current_df)
    COUNTS_UNIQUE_MOLECULES['Units'] = current_df['InchiKey'].nunique() if 'InchiKey' in current_df.columns else current_df['canonical_smiles'].nunique()
    
    # Relation filter
    current_df = current_df[current_df['standard_relation'] == '=']
    COUNTS_TOTAL_ACTIVITIES['Relation'] = len(current_df)
    COUNTS_UNIQUE_MOLECULES['Relation'] = current_df['InchiKey'].nunique() if 'InchiKey' in current_df.columns else current_df['canonical_smiles'].nunique()
    
    # Validity filter
    current_df = current_df[current_df['data_validity_comment'].isnull()]
    COUNTS_TOTAL_ACTIVITIES['Validity'] = len(current_df)
    COUNTS_UNIQUE_MOLECULES['Validity'] = current_df['InchiKey'].nunique() if 'InchiKey' in current_df.columns else current_df['canonical_smiles'].nunique()
    
    # Confidence filter
    current_df['assay_confidence_score'] = pd.to_numeric(current_df['assay_confidence_score'], errors='coerce')
    current_df = current_df[current_df['assay_confidence_score'] == 9]
    COUNTS_TOTAL_ACTIVITIES['Confidence'] = len(current_df)
    COUNTS_UNIQUE_MOLECULES['Confidence'] = current_df['InchiKey'].nunique() if 'InchiKey' in current_df.columns else current_df['canonical_smiles'].nunique()
    
    # Docs filter
    current_df = current_df[current_df['document_year'].notnull()]
    COUNTS_TOTAL_ACTIVITIES['Docs'] = len(current_df)
    COUNTS_UNIQUE_MOLECULES['Docs'] = current_df['InchiKey'].nunique() if 'InchiKey' in current_df.columns else current_df['canonical_smiles'].nunique()
    
    # Mutant filter
    mutant_keywords = ['mutant', 'mutation', 'variant']
    mask = current_df['assay_description'].astype(str).str.lower().apply(
        lambda x: any(k in x for k in mutant_keywords)
    )
    current_df = current_df[~mask]
    COUNTS_TOTAL_ACTIVITIES['No Mutants'] = len(current_df)
    COUNTS_UNIQUE_MOLECULES['No Mutants'] = current_df['InchiKey'].nunique() if 'InchiKey' in current_df.columns else current_df['canonical_smiles'].nunique()
    
    # Assay Size filter
    minAssaySize = 10
    maxAssaySize = 3000
    assay_counts = current_df.groupby('assay_chembl_id').size()
    valid_assays = assay_counts[(assay_counts >= minAssaySize) & (assay_counts <= maxAssaySize)].index
    current_df = current_df[current_df['assay_chembl_id'].isin(valid_assays)]
    COUNTS_TOTAL_ACTIVITIES['Assay Size'] = len(current_df)
    COUNTS_UNIQUE_MOLECULES['Assay Size'] = current_df['InchiKey'].nunique() if 'InchiKey' in current_df.columns else current_df['canonical_smiles'].nunique()
    
    print("\n✅ Computed counts from raw data:")
    print("-" * 60)
    print(f"{'Step':<15} {'Total Activities':>20} {'Unique Molecules':>20}")
    print("-" * 60)
    for step in COUNTS_TOTAL_ACTIVITIES.keys():
        print(f"{step:<15} {COUNTS_TOTAL_ACTIVITIES[step]:>20,} {COUNTS_UNIQUE_MOLECULES[step]:>20,}")
    
else:
    print(f"⚠ Raw parquet not found at {raw_path}")
    print("  Please ensure the raw data file exists.")

# Now load overlap data and add those counts
overlap_data = {}

if minoverlap_path.exists():
    overlap_data['min'] = pd.read_parquet(minoverlap_path)
    min_unique = overlap_data['min']['InchiKey'].nunique() if 'InchiKey' in overlap_data['min'].columns else len(overlap_data['min'])
    COUNTS_TOTAL_ACTIVITIES['Min Overlap'] = len(overlap_data['min'])
    COUNTS_UNIQUE_MOLECULES['Min Overlap'] = min_unique
    print(f"\n✓ Loaded minimal overlap data: {len(overlap_data['min']):,} rows, {min_unique:,} unique molecules")
else:
    print(f"\n⚠ Min overlap file not found: {minoverlap_path}")

if maxoverlap_path.exists():
    overlap_data['max'] = pd.read_parquet(maxoverlap_path)
    max_unique = overlap_data['max']['InchiKey'].nunique() if 'InchiKey' in overlap_data['max'].columns else len(overlap_data['max'])
    COUNTS_TOTAL_ACTIVITIES['Max Overlap'] = len(overlap_data['max'])
    COUNTS_UNIQUE_MOLECULES['Max Overlap'] = max_unique
    print(f"✓ Loaded maximal overlap data: {len(overlap_data['max']):,} rows, {max_unique:,} unique molecules")
else:
    print(f"⚠ Max overlap file not found: {maxoverlap_path}")

# Print final summary
print("\n" + "=" * 60)
print("FINAL COUNT SUMMARY")
print("=" * 60)
print(f"{'Step':<15} {'Total Activities':>20} {'Unique Molecules':>20}")
print("-" * 60)
for step in COUNTS_TOTAL_ACTIVITIES.keys():
    print(f"{step:<15} {COUNTS_TOTAL_ACTIVITIES[step]:>20,} {COUNTS_UNIQUE_MOLECULES[step]:>20,}")
print("=" * 60)

Loading raw data to compute actual counts...

✅ Computed counts from raw data:
------------------------------------------------------------
Step                Total Activities     Unique Molecules
------------------------------------------------------------
Initial                    1,237,090              565,647
Type                       1,051,008              517,057
Units                        983,284              508,301
Relation                     749,185              429,105
Validity                     733,391              421,605
Confidence                   420,967              273,193
Docs                         415,109              268,959
No Mutants                   405,562              264,210
Assay Size                   333,748              237,873

✅ Computed counts from raw data:
------------------------------------------------------------
Step                Total Activities     Unique Molecules
------------------------------------------------------------
Initi

In [7]:
import plotly.graph_objects as go
import plotly.io as pio
from pathlib import Path

# Ensure output directory exists
output_dir = Path("../data/output/figures")
output_dir.mkdir(parents=True, exist_ok=True)

# ============================================================
# HELPER FUNCTION: Save Plotly figures in all publication formats
# ============================================================
def save_plotly_figure(fig, output_base, scale=3):
    """
    Save a Plotly figure in multiple publication-ready formats.
    
    Saves:
    - SVG (vector format, ideal for publication)
    - PNG (high-resolution raster, scale=3 for 3x resolution)
    - PDF (vector format for print)
    - HTML (interactive version)
    
    Args:
        fig: Plotly figure object
        output_base: Path or string base path (without extension)
        scale: Resolution scale for PNG (default=3)
    
    Returns:
        dict with paths of successfully saved files
    """
    out_base = Path(output_base)
    saved = {}
    
    # Try to save vector/raster formats (requires kaleido)
    try:
        # SVG - vector format, best for publications
        fig.write_image(str(out_base) + ".svg")
        print(f"✓ Saved: {out_base}.svg")
        saved['svg'] = str(out_base) + ".svg"
        
        # PNG - high-resolution raster
        fig.write_image(str(out_base) + ".png", scale=scale)
        print(f"✓ Saved: {out_base}.png")
        saved['png'] = str(out_base) + ".png"
        
        # PDF - vector format for print
        fig.write_image(str(out_base) + ".pdf")
        print(f"✓ Saved: {out_base}.pdf")
        saved['pdf'] = str(out_base) + ".pdf"
        
    except ValueError as e:
        if "kaleido" in str(e).lower():
            print("⚠ Kaleido not installed - skipping SVG/PNG/PDF export.")
            print("  Install with: pip install -U kaleido")
        else:
            raise
    
    # HTML - always works, no external dependencies
    fig.write_html(str(out_base) + ".html", include_plotlyjs='cdn')
    print(f"✓ Saved: {out_base}.html")
    saved['html'] = str(out_base) + ".html"
    
    return saved

# ============================================================
# OPTION 2: Total Activities (rows) - what you see in the filtered parquet
# These need to be computed from actual data or entered manually
# Based on your observation: final = 280,028 total activities
# ============================================================
# You can either:
# A) Run the filtering notebook and extract len(df) at each step, OR
# B) Load the filtered parquet and count rows

# Let's load the actual filtered data to get the total activities count
import pandas as pd

filtered_path = Path("../data/filtered_chembl_affinity.parquet")

In [8]:
def create_publication_bar_chart(
    counts: dict,
    title: str,
    output_basename: str,
    ylabel: str = "Count",
    highlight_first_color: str = "#2E86AB",  # Teal blue for initial
    highlight_last_color: str = "#A23B72",   # Magenta for final
    bar_color: str = "#4A90A4",              # Muted steel blue for middle bars
) -> go.Figure:
    """
    Create a publication-ready bar chart with Plotly.
    
    - Larger fonts for readability
    - Highlighted first and last bars with distinct colors
    - Clean, professional styling
    - Saves SVG, PNG, PDF (if kaleido available), and interactive HTML
    """
    steps = list(counts.keys())
    values = list(counts.values())
    n = len(steps)
    
    # Create color list: highlight first and last
    colors = [bar_color] * n
    colors[0] = highlight_first_color
    colors[-1] = highlight_last_color
    
    # Create text color list for annotations (matching bar colors for emphasis)
    text_colors = ['#333333'] * n
    text_colors[0] = highlight_first_color
    text_colors[-1] = highlight_last_color
    
    # Create the figure
    fig = go.Figure()
    
    # Add bars
    fig.add_trace(go.Bar(
        x=steps,
        y=values,
        marker=dict(
            color=colors,
            line=dict(color='#2C3E50', width=1.5),
        ),
        text=[f'{v:,}' for v in values],
        textposition='outside',
        textfont=dict(
            size=16,
            color=text_colors,
            family='Arial Black',
        ),
        hovertemplate='<b>%{x}</b><br>' + ylabel + ': %{y:,}<extra></extra>',
    ))
    
    # Update layout for publication quality
    fig.update_layout(
        title=dict(
            text=title,
            font=dict(size=22, family='Arial', color='#1a1a1a'),
            x=0.5,
            xanchor='center',
        ),
        xaxis=dict(
            title=dict(text='Filtering Step', font=dict(size=18, family='Arial')),
            tickfont=dict(size=14, family='Arial'),
            tickangle=-35,
            showgrid=False,
            linecolor='#2C3E50',
            linewidth=2,
        ),
        yaxis=dict(
            title=dict(text=ylabel, font=dict(size=18, family='Arial')),
            tickfont=dict(size=14, family='Arial'),
            tickformat=',',
            showgrid=True,
            gridcolor='rgba(200,200,200,0.3)',
            gridwidth=1,
            linecolor='#2C3E50',
            linewidth=2,
            zeroline=True,
            zerolinecolor='#2C3E50',
            zerolinewidth=2,
        ),
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=80, r=40, t=100, b=120),
        width=1200,
        height=600,
        showlegend=False,
        bargap=0.25,
    )
    
    # Adjust y-axis range to fit text labels
    max_val = max(values)
    fig.update_yaxes(range=[0, max_val * 1.15])
    
    # Save outputs using centralized helper (includes SVG)
    out_base = output_dir / output_basename
    save_plotly_figure(fig, out_base, scale=3)
    
    return fig


# ============================================================
# Prepare data subsets (with/without overlap)
# ===========================================================

# Unique molecules version
counts_unique_without_overlap = {k: COUNTS_UNIQUE_MOLECULES[k] for k in [
    'Initial', 'Type', 'Units', 'Relation', 'Validity', 
    'Confidence', 'Docs', 'No Mutants', 'Assay Size'
]}

counts_unique_with_overlap = {k: COUNTS_UNIQUE_MOLECULES[k] for k in [
    'Initial', 'Type', 'Units', 'Relation', 'Validity', 
    'Confidence', 'Docs', 'No Mutants', 'Assay Size', 
    'Min Overlap', 'Max Overlap'
]}

# Total activities version
counts_activities_without_overlap = {k: COUNTS_TOTAL_ACTIVITIES[k] for k in [
    'Initial', 'Type', 'Units', 'Relation', 'Validity', 
    'Confidence', 'Docs', 'No Mutants', 'Assay Size'
]}

counts_activities_with_overlap = {k: COUNTS_TOTAL_ACTIVITIES[k] for k in [
    'Initial', 'Type', 'Units', 'Relation', 'Validity', 
    'Confidence', 'Docs', 'No Mutants', 'Assay Size', 
    'Min Overlap', 'Max Overlap'
]}

print("Data subsets prepared for plotting.")

Data subsets prepared for plotting.


## Figure 1: Unique Molecules (Without Overlap Filtering)

Shows unique molecules (by InchiKey) from Initial (566,342) to Assay Size (237,987)

In [9]:
# Generate Figure 1: Unique Molecules - Without overlap filtering
fig1 = create_publication_bar_chart(
    counts=counts_unique_without_overlap,
    title="Unique Molecules Remaining After Each Filtering Step",
    output_basename="unique_molecules_without_overlap",
    ylabel="Number of Unique Molecules",
)
fig1.show()

✓ Saved: ../data/output/figures/unique_molecules_without_overlap.svg
✓ Saved: ../data/output/figures/unique_molecules_without_overlap.png
✓ Saved: ../data/output/figures/unique_molecules_without_overlap.pdf
✓ Saved: ../data/output/figures/unique_molecules_without_overlap.html


## Figure 2: Unique Molecules (With Overlap Filtering)

Shows unique molecules from Initial (566,342) through Max Overlap (11,099)

In [10]:
# Generate Figure 2: Unique Molecules - With overlap filtering
fig2 = create_publication_bar_chart(
    counts=counts_unique_with_overlap,
    title="Unique Molecules Remaining After Each Filtering Step (Including Overlap)",
    output_basename="unique_molecules_with_overlap",
    ylabel="Number of Unique Molecules",
)
fig2.show()

✓ Saved: ../data/output/figures/unique_molecules_with_overlap.svg
✓ Saved: ../data/output/figures/unique_molecules_with_overlap.png
✓ Saved: ../data/output/figures/unique_molecules_with_overlap.png
✓ Saved: ../data/output/figures/unique_molecules_with_overlap.pdf
✓ Saved: ../data/output/figures/unique_molecules_with_overlap.html
✓ Saved: ../data/output/figures/unique_molecules_with_overlap.pdf
✓ Saved: ../data/output/figures/unique_molecules_with_overlap.html


## Figure 3: Total Activities (Without Overlap Filtering)

Shows total activity records (rows) rather than unique molecules.  
**Note:** Update the `COUNTS_TOTAL_ACTIVITIES` dict with actual counts from your filtering notebook!

In [26]:
counts_activities_without_overlap = {'Initial': 693892,
    'Type': 634500,
    'Units': 622000,
    'Relation': 525000,
    'Validity': 515000,
    'Confidence': 334000,
    'Docs': 329000,
    'No Mutants': 323000,
    'Assay Size': 280028
    }

In [27]:
# Generate Figure 3: Total Activities - Without overlap filtering
fig3 = create_publication_bar_chart(
    counts=counts_activities_without_overlap,
    title="Total Activity Records Remaining After Each Filtering Step",
    output_basename="total_activities_without_overlap",
    ylabel="Number of Activity Records",
)
fig3.show()

✓ Saved: ../data/output/figures/total_activities_without_overlap.svg
✓ Saved: ../data/output/figures/total_activities_without_overlap.png
✓ Saved: ../data/output/figures/total_activities_without_overlap.png
✓ Saved: ../data/output/figures/total_activities_without_overlap.pdf
✓ Saved: ../data/output/figures/total_activities_without_overlap.html
✓ Saved: ../data/output/figures/total_activities_without_overlap.pdf
✓ Saved: ../data/output/figures/total_activities_without_overlap.html


## Summary Statistics

In [12]:
# Print summary statistics comparing both metrics
print("=" * 70)
print("FILTERING SUMMARY: UNIQUE MOLECULES vs TOTAL ACTIVITIES")
print("=" * 70)

print("\n📊 Unique Molecules (by InchiKey):")
print(f"   Initial:      {COUNTS_UNIQUE_MOLECULES['Initial']:>12,}")
print(f"   After filters:{COUNTS_UNIQUE_MOLECULES['Assay Size']:>12,}")
print(f"   Retention:    {COUNTS_UNIQUE_MOLECULES['Assay Size']/COUNTS_UNIQUE_MOLECULES['Initial']*100:>11.1f}%")

print("\n📊 Total Activities (rows):")
print(f"   Initial:      {COUNTS_TOTAL_ACTIVITIES['Initial']:>12,}")
print(f"   After filters:{COUNTS_TOTAL_ACTIVITIES['Assay Size']:>12,}")
print(f"   Retention:    {COUNTS_TOTAL_ACTIVITIES['Assay Size']/COUNTS_TOTAL_ACTIVITIES['Initial']*100:>11.1f}%")

print("\n📊 Average activities per unique molecule:")
avg_before = COUNTS_TOTAL_ACTIVITIES['Initial'] / COUNTS_UNIQUE_MOLECULES['Initial']
avg_after = COUNTS_TOTAL_ACTIVITIES['Assay Size'] / COUNTS_UNIQUE_MOLECULES['Assay Size']
print(f"   Before filtering: {avg_before:.2f}")
print(f"   After filtering:  {avg_after:.2f}")

print("\n" + "=" * 70)
print(f"Output files saved to: {output_dir.absolute()}")
print("=" * 70)

FILTERING SUMMARY: UNIQUE MOLECULES vs TOTAL ACTIVITIES

📊 Unique Molecules (by InchiKey):
   Initial:           565,647
   After filters:     237,873
   Retention:           42.1%

📊 Total Activities (rows):
   Initial:         1,237,090
   After filters:     333,748
   Retention:           27.0%

📊 Average activities per unique molecule:
   Before filtering: 2.19
   After filtering:  1.40

Output files saved to: /Users/aoxu/projects/PLATE-VS/notebooks/../data/output/figures


## Figure 4: Distribution of Actives Across UniProt Targets

Publication-ready visualization showing the number of actives per UniProt target:
- Sorted by number of actives (descending)
- Color gradient to show magnitude
- Interactive Plotly visualization

In [13]:
# Publication-ready distribution of actives per UniProt target
# Using the counts_df that was already loaded earlier

# Sort by number of actives (descending) for the full dataset
sorted_df = counts_df.sort_values('n_actives', ascending=False).reset_index(drop=True)

# Create color scale based on number of actives
import plotly.express as px

# Option 1: Full bar chart (all UniProts) - good for overview
fig_dist = go.Figure()

# Add bars with color gradient
fig_dist.add_trace(go.Bar(
    x=sorted_df['uniprot_id'],
    y=sorted_df['n_actives'],
    marker=dict(
        color=sorted_df['n_actives'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(
            title=dict(
                text='Actives',
                font=dict(size=18, family='Arial'),
            ),
            tickfont=dict(size=16, family='Arial'),
        ),
        line=dict(width=0),
    ),
    hovertemplate='<b>%{x}</b><br>Actives: %{y:,}<extra></extra>',
))

# Update layout
fig_dist.update_layout(
    title=dict(
        text='Distribution of Actives Across UniProt Targets',
        font=dict(size=22, family='Arial', color='#1a1a1a'),
        x=0.5,
        xanchor='center',
    ),
    xaxis=dict(
        title=dict(text='UniProt Target', font=dict(size=18, family='Arial')),
        tickfont=dict(size=8, family='Arial'),
        tickangle=-90,
        showticklabels=False,  # Hide labels due to too many targets
        showgrid=False,
        linecolor='#2C3E50',
        linewidth=2,
    ),
    yaxis=dict(
        title=dict(text='Number of Actives', font=dict(size=18, family='Arial')),
        tickfont=dict(size=16, family='Arial'),
        tickformat=',',
        showgrid=True,
        gridcolor='rgba(200,200,200,0.3)',
        gridwidth=1,
        linecolor='#2C3E50',
        linewidth=2,
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=80, r=100, t=100, b=60),
    width=1400,
    height=600,
    showlegend=False,
)

# Save outputs using centralized helper (includes SVG)
out_base = output_dir / "actives_distribution_by_uniprot"
save_plotly_figure(fig_dist, out_base, scale=3)

fig_dist.show()

✓ Saved: ../data/output/figures/actives_distribution_by_uniprot.svg
✓ Saved: ../data/output/figures/actives_distribution_by_uniprot.png
✓ Saved: ../data/output/figures/actives_distribution_by_uniprot.png
✓ Saved: ../data/output/figures/actives_distribution_by_uniprot.pdf
✓ Saved: ../data/output/figures/actives_distribution_by_uniprot.html
✓ Saved: ../data/output/figures/actives_distribution_by_uniprot.pdf
✓ Saved: ../data/output/figures/actives_distribution_by_uniprot.html


## Figure 5: Top 50 UniProt Targets by Number of Actives

A more readable version showing only the top 50 targets with labels.

In [14]:
# Top 50 UniProt targets - with visible labels
top_n = 50
top_df = sorted_df.head(top_n)

# Create colors: highlight top 3 targets
colors_top = ['#E63946'] * 3 + ['#457B9D'] * (top_n - 3)

fig_top = go.Figure()

fig_top.add_trace(go.Bar(
    x=top_df['uniprot_id'],
    y=top_df['n_actives'],
    marker=dict(
        color=colors_top,
        line=dict(color='#1D3557', width=1),
    ),
    text=[f'{v:,}' for v in top_df['n_actives']],
    textposition='outside',
    textfont=dict(size=15, family='Arial', color='#333'),
    hovertemplate='<b>%{x}</b><br>Actives: %{y:,}<extra></extra>',
))

fig_top.update_layout(
    title=dict(
        text=f'Top {top_n} UniProt Targets by Number of Actives',
        font=dict(size=30, family='Arial', color='#1a1a1a'),
        x=0.5,
        xanchor='center',
    ),
    xaxis=dict(
        title=dict(text='UniProt Target', font=dict(size=18, family='Arial')),
        tickfont=dict(size=10, family='Arial'),
        tickangle=-45,
        showgrid=False,
        linecolor='#2C3E50',
        linewidth=2,
    ),
    yaxis=dict(
        title=dict(text='Number of Actives', font=dict(size=18, family='Arial')),
        tickfont=dict(size=18, family='Arial'),
        tickformat=',',
        showgrid=True,
        gridcolor='rgba(200,200,200,0.3)',
        gridwidth=1,
        linecolor='#2C3E50',
        linewidth=2,
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=80, r=40, t=100, b=150),
    width=1400,
    height=700,
    showlegend=False,
)

# Adjust y-axis to fit text labels
fig_top.update_yaxes(range=[0, top_df['n_actives'].max() * 1.15])

# Save outputs using centralized helper (includes SVG)
out_base = output_dir / f"top_{top_n}_uniprot_actives"
save_plotly_figure(fig_top, out_base, scale=3)

fig_top.show()

# Print top 10 for reference
print(f"\n📊 Top 10 UniProt Targets:")
print("-" * 40)
for i, (_, row) in enumerate(top_df.head(10).iterrows(), 1):
    print(f"  {i:2}. {row['uniprot_id']:12} : {row['n_actives']:,} actives")

✓ Saved: ../data/output/figures/top_50_uniprot_actives.svg
✓ Saved: ../data/output/figures/top_50_uniprot_actives.png
✓ Saved: ../data/output/figures/top_50_uniprot_actives.png
✓ Saved: ../data/output/figures/top_50_uniprot_actives.pdf
✓ Saved: ../data/output/figures/top_50_uniprot_actives.html
✓ Saved: ../data/output/figures/top_50_uniprot_actives.pdf
✓ Saved: ../data/output/figures/top_50_uniprot_actives.html



📊 Top 10 UniProt Targets:
----------------------------------------
   1. O00763       : 2,873 actives
   2. P07339       : 2,820 actives
   3. Q9Y233       : 2,793 actives
   4. P07711       : 2,791 actives
   5. P00734       : 2,775 actives
   6. P03956       : 2,743 actives
   7. Q9NWZ3       : 2,743 actives
   8. P20248       : 2,741 actives
   9. P04629       : 2,726 actives
  10. P11309       : 2,718 actives


## Figure 6: Bottom 50 UniProt Targets by Number of Actives

Showing the 50 targets with the fewest actives (excluding targets with <5 actives).

In [21]:
# Bottom 50 UniProt targets - with visible labels (excluding <5 actives)
min_actives = 10
bottom_n = 50

# Filter out targets with fewer than 5 actives, then get bottom 50
filtered_sorted_df = sorted_df[sorted_df['n_actives'] >= min_actives].copy()
bottom_df = filtered_sorted_df.tail(bottom_n).iloc[::-1]  # Reverse to show smallest first

print(f"Total targets with ≥{min_actives} actives: {len(filtered_sorted_df)}")
print(f"Showing bottom {len(bottom_df)} targets")

# Create colors: highlight bottom 3 targets (fewest actives)
colors_bottom = ['#F4A261'] * 3 + ['#2A9D8F'] * (len(bottom_df) - 3)

fig_bottom = go.Figure()

fig_bottom.add_trace(go.Bar(
    x=bottom_df['uniprot_id'],
    y=bottom_df['n_actives'],
    marker=dict(
        color=colors_bottom,
        line=dict(color='#264653', width=1),
    ),
    text=[f'{v:,}' for v in bottom_df['n_actives']],
    textposition='outside',
    textfont=dict(size=15, family='Arial', color='#333'),
    hovertemplate='<b>%{x}</b><br>Actives: %{y:,}<extra></extra>',
))

fig_bottom.update_layout(
    title=dict(
        text=f'Bottom {bottom_n} UniProt Targets by Number of Actives (≥{min_actives} actives)',
        font=dict(size=30, family='Arial', color='#1a1a1a'),
        x=0.5,
        xanchor='center',
    ),
    xaxis=dict(
        title=dict(text='UniProt Target', font=dict(size=18, family='Arial')),
        tickfont=dict(size=10, family='Arial'),
        tickangle=-45,
        showgrid=False,
        linecolor='#2C3E50',
        linewidth=2,
    ),
    yaxis=dict(
        title=dict(text='Number of Actives', font=dict(size=18, family='Arial')),
        tickfont=dict(size=18, family='Arial'),
        tickformat=',',
        showgrid=True,
        gridcolor='rgba(200,200,200,0.3)',
        gridwidth=1,
        linecolor='#2C3E50',
        linewidth=2,
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=80, r=40, t=100, b=150),
    width=1400,
    height=700,
    showlegend=False,
)

# Adjust y-axis to fit text labels
fig_bottom.update_yaxes(range=[0, bottom_df['n_actives'].max() * 1.25])

# Save outputs using centralized helper (includes SVG)
out_base = output_dir / f"bottom_{bottom_n}_uniprot_actives"
save_plotly_figure(fig_bottom, out_base, scale=3)

fig_bottom.show()

# Print bottom 10 for reference
print(f"\n📊 Bottom 10 UniProt Targets (with ≥{min_actives} actives):")
print("-" * 40)
for i, (_, row) in enumerate(bottom_df.head(10).iterrows(), 1):
    print(f"  {i:2}. {row['uniprot_id']:12} : {row['n_actives']:,} actives")

Total targets with ≥10 actives: 1146
Showing bottom 50 targets
✓ Saved: ../data/output/figures/bottom_50_uniprot_actives.svg
✓ Saved: ../data/output/figures/bottom_50_uniprot_actives.svg
✓ Saved: ../data/output/figures/bottom_50_uniprot_actives.png
✓ Saved: ../data/output/figures/bottom_50_uniprot_actives.png
✓ Saved: ../data/output/figures/bottom_50_uniprot_actives.pdf
✓ Saved: ../data/output/figures/bottom_50_uniprot_actives.html
✓ Saved: ../data/output/figures/bottom_50_uniprot_actives.pdf
✓ Saved: ../data/output/figures/bottom_50_uniprot_actives.html



📊 Bottom 10 UniProt Targets (with ≥10 actives):
----------------------------------------
   1. Q9Y2Q3       : 10 actives
   2. Q92997       : 10 actives
   3. P29965       : 10 actives
   4. Q96GX5       : 10 actives
   5. P01031       : 10 actives
   6. Q9NP87       : 10 actives
   7. P68871       : 10 actives
   8. P35475       : 10 actives
   9. P19623       : 10 actives
  10. P49638       : 10 actives


## Figure 7: Overlap Filtering - pChEMBL Agreement Analysis

Publication-ready visualization of the overlap filtering results showing:
- **Minimal Curation**: Only requires activity overlap
- **Maximal Curation**: Requires metadata match (assay type, cell line, etc.)

For each curation level:
- Left: Scatter plot of pChEMBL values between assay pairs
- Right: Histogram of absolute differences

In [16]:
# Load overlap filtering data
from scipy import stats
from sklearn.metrics import r2_score, median_absolute_error
from plotly.subplots import make_subplots

# Try to load the overlap parquet files
minoverlap_path = Path("../data/minoverlap_chembl_affinity.parquet")
maxoverlap_path = Path("../data/maxoverlap_chembl_affinity.parquet")

overlap_data = {}

if minoverlap_path.exists():
    overlap_data['min'] = pd.read_parquet(minoverlap_path)
    print(f"✓ Loaded minimal overlap data: {len(overlap_data['min']):,} rows")
else:
    print(f"⚠ Min overlap file not found: {minoverlap_path}")

if maxoverlap_path.exists():
    overlap_data['max'] = pd.read_parquet(maxoverlap_path)
    print(f"✓ Loaded maximal overlap data: {len(overlap_data['max']):,} rows")
else:
    print(f"⚠ Max overlap file not found: {maxoverlap_path}")

# Check column names
if 'min' in overlap_data:
    print(f"\nMin overlap columns: {list(overlap_data['min'].columns)[:15]}...")
if 'max' in overlap_data:
    print(f"Max overlap columns: {list(overlap_data['max'].columns)[:15]}...")

✓ Loaded minimal overlap data: 63,589 rows
✓ Loaded maximal overlap data: 45,379 rows

Min overlap columns: ['action_type_1', 'activity_comment_1', 'activity_id_1', 'activity_properties_1', 'assay_chembl_id_1', 'assay_description_1', 'assay_type_1', 'assay_variant_accession_1', 'assay_variant_mutation_1', 'bao_endpoint_1', 'bao_format_1', 'bao_label_1', 'canonical_smiles_1', 'data_validity_comment_1', 'data_validity_description_1']...
Max overlap columns: ['action_type_1', 'activity_comment_1', 'activity_id_1', 'activity_properties_1', 'assay_chembl_id_1', 'assay_description_1', 'assay_type_1', 'assay_variant_accession_1', 'assay_variant_mutation_1', 'bao_endpoint_1', 'bao_format_1', 'bao_label_1', 'canonical_smiles_1', 'data_validity_comment_1', 'data_validity_description_1']...


In [17]:
def create_agreement_figure(df, title, output_basename, pchembl_col1='pchembl_value_1', pchembl_col2='pchembl_value_2'):
    """
    Create publication-ready agreement plots using Plotly.
    
    Returns: figure with scatter plot (left) and difference histogram (right)
    Saves: SVG, PNG, PDF (if kaleido available), and HTML
    """
    if df.empty:
        print(f"No data to plot for {title}.")
        return None
    
    # Try to find pchembl columns
    if pchembl_col1 not in df.columns or pchembl_col2 not in df.columns:
        # Try alternative column names
        pchembl_cols = [c for c in df.columns if 'pchembl' in c.lower()]
        if len(pchembl_cols) >= 2:
            pchembl_col1, pchembl_col2 = pchembl_cols[0], pchembl_cols[1]
            print(f"Using columns: {pchembl_col1}, {pchembl_col2}")
        else:
            print(f"Could not find pchembl columns. Available: {df.columns.tolist()}")
            return None
    
    xp = df[pchembl_col1].values
    yp = df[pchembl_col2].values
    delts = np.abs(xp - yp)
    
    # Calculate statistics
    r, p_val = stats.spearmanr(xp, yp)
    r2 = r2_score(xp, yp)
    mae = median_absolute_error(xp, yp)
    npts = len(delts)
    count_03 = np.sum(delts > 0.3)
    frac_03 = count_03 / npts
    count_10 = np.sum(delts > 1.0)
    frac_10 = count_10 / npts
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(f'{title} - Correlation', f'{title} - Difference Histogram'),
        horizontal_spacing=0.12,
    )
    
    # Left: Scatter plot
    fig.add_trace(
        go.Scattergl(
            x=xp, y=yp,
            mode='markers',
            marker=dict(
                size=4,
                color='#3498db',
                opacity=0.3,
            ),
            name='Data points',
            hovertemplate='Assay 1: %{x:.2f}<br>Assay 2: %{y:.2f}<extra></extra>',
        ),
        row=1, col=1
    )
    
    # Add reference lines
    line_min, line_max = 4, 11
    # x=y line
    fig.add_trace(
        go.Scatter(x=[line_min, line_max], y=[line_min, line_max],
                   mode='lines', line=dict(color='black', width=2),
                   name='x=y', showlegend=True),
        row=1, col=1
    )
    # ±0.3 lines
    fig.add_trace(
        go.Scatter(x=[line_min, line_max], y=[line_min-0.3, line_max-0.3],
                   mode='lines', line=dict(color='#E67E22', width=1.5, dash='dash'),
                   name='±0.3', showlegend=True),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=[line_min, line_max], y=[line_min+0.3, line_max+0.3],
                   mode='lines', line=dict(color='#E67E22', width=1.5, dash='dash'),
                   showlegend=False),
        row=1, col=1
    )
    # ±1.0 lines
    fig.add_trace(
        go.Scatter(x=[line_min, line_max], y=[line_min-1.0, line_max-1.0],
                   mode='lines', line=dict(color='#E74C3C', width=1.5, dash='dot'),
                   name='±1.0', showlegend=True),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=[line_min, line_max], y=[line_min+1.0, line_max+1.0],
                   mode='lines', line=dict(color='#E74C3C', width=1.5, dash='dot'),
                   showlegend=False),
        row=1, col=1
    )
    
    # Right: Histogram of differences
    fig.add_trace(
        go.Histogram(
            x=delts,
            nbinsx=30,
            marker=dict(
                color='#2ECC71',
                line=dict(color='#1D8348', width=1),
            ),
            name='Δ pChEMBL',
            hovertemplate='Δ: %{x:.2f}<br>Count: %{y}<extra></extra>',
        ),
        row=1, col=2
    )
    
    # Add vertical lines for thresholds
    fig.add_vline(x=0.3, line=dict(color='#E67E22', width=2, dash='dash'), 
                  annotation_text='0.3', row=1, col=2)
    fig.add_vline(x=1.0, line=dict(color='#E74C3C', width=2, dash='dot'),
                  annotation_text='1.0', row=1, col=2)
    
    # Update layout
    fig.update_layout(
        title=dict(
            text=f'{title}<br><sup>n={npts:,} | R²={r2:.3f} | Spearman ρ={r:.3f} | MAE={mae:.3f} | >{" "}0.3: {frac_03:.1%} | >{" "}1.0: {frac_10:.1%}</sup>',
            font=dict(size=20, family='Arial'),
            x=0.5,
            xanchor='center',
        ),
        plot_bgcolor='white',
        paper_bgcolor='white',
        width=1400,
        height=550,
        showlegend=True,
        legend=dict(
            x=0.01, y=0.99,
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='#ccc',
            borderwidth=1,
        ),
    )
    
    # Update axes for scatter plot
    fig.update_xaxes(
        title_text='Assay 1 pChEMBL', title_font=dict(size=16),
        tickfont=dict(size=14), range=[3.5, 11.5],
        showgrid=True, gridcolor='rgba(200,200,200,0.3)',
        linecolor='#2C3E50', linewidth=2,
        row=1, col=1
    )
    fig.update_yaxes(
        title_text='Assay 2 pChEMBL', title_font=dict(size=16),
        tickfont=dict(size=14), range=[3.5, 11.5],
        showgrid=True, gridcolor='rgba(200,200,200,0.3)',
        linecolor='#2C3E50', linewidth=2,
        row=1, col=1
    )
    
    # Update axes for histogram
    fig.update_xaxes(
        title_text='Δ pChEMBL (absolute difference)', title_font=dict(size=16),
        tickfont=dict(size=14),
        showgrid=False, linecolor='#2C3E50', linewidth=2,
        row=1, col=2
    )
    fig.update_yaxes(
        title_text='Count', title_font=dict(size=16),
        tickfont=dict(size=14),
        showgrid=True, gridcolor='rgba(200,200,200,0.3)',
        linecolor='#2C3E50', linewidth=2,
        row=1, col=2
    )
    
    # Save outputs using centralized helper (includes SVG)
    out_base = output_dir / output_basename
    save_plotly_figure(fig, out_base, scale=3)
    
    # Print statistics
    print(f"\n📊 Statistics for {title}:")
    print(f"   Number of pairs: {npts:,}")
    print(f"   R²: {r2:.4f}")
    print(f"   Spearman ρ: {r:.4f}")
    print(f"   Median Absolute Error: {mae:.4f}")
    print(f"   Points with Δ > 0.3: {count_03:,} ({frac_03:.2%})")
    print(f"   Points with Δ > 1.0: {count_10:,} ({frac_10:.2%})")
    
    return fig

In [18]:
# Generate agreement plot for Minimal Curation
if 'min' in overlap_data:
    fig_min_agreement = create_agreement_figure(
        overlap_data['min'],
        title="Minimal Curation (Activity Overlap Only)",
        output_basename="minimal_curation_agreement",
    )
    if fig_min_agreement:
        fig_min_agreement.show()
else:
    print("⚠ No minimal overlap data available")

✓ Saved: ../data/output/figures/minimal_curation_agreement.svg
✓ Saved: ../data/output/figures/minimal_curation_agreement.png
✓ Saved: ../data/output/figures/minimal_curation_agreement.png
✓ Saved: ../data/output/figures/minimal_curation_agreement.pdf
✓ Saved: ../data/output/figures/minimal_curation_agreement.html

📊 Statistics for Minimal Curation (Activity Overlap Only):
   Number of pairs: 63,589
   R²: 0.8016
   Spearman ρ: 0.8969
   Median Absolute Error: 0.0000
   Points with Δ > 0.3: 14,579 (22.93%)
   Points with Δ > 1.0: 4,776 (7.51%)
✓ Saved: ../data/output/figures/minimal_curation_agreement.pdf
✓ Saved: ../data/output/figures/minimal_curation_agreement.html

📊 Statistics for Minimal Curation (Activity Overlap Only):
   Number of pairs: 63,589
   R²: 0.8016
   Spearman ρ: 0.8969
   Median Absolute Error: 0.0000
   Points with Δ > 0.3: 14,579 (22.93%)
   Points with Δ > 1.0: 4,776 (7.51%)


In [19]:
# Generate agreement plot for Maximal Curation
if 'max' in overlap_data:
    fig_max_agreement = create_agreement_figure(
        overlap_data['max'],
        title="Maximal Curation (Metadata Match)",
        output_basename="maximal_curation_agreement",
    )
    if fig_max_agreement:
        fig_max_agreement.show()
else:
    print("⚠ No maximal overlap data available")

✓ Saved: ../data/output/figures/maximal_curation_agreement.svg
✓ Saved: ../data/output/figures/maximal_curation_agreement.png
✓ Saved: ../data/output/figures/maximal_curation_agreement.png
✓ Saved: ../data/output/figures/maximal_curation_agreement.pdf
✓ Saved: ../data/output/figures/maximal_curation_agreement.html

📊 Statistics for Maximal Curation (Metadata Match):
   Number of pairs: 45,379
   R²: 0.8780
   Spearman ρ: 0.9377
   Median Absolute Error: 0.0000
   Points with Δ > 0.3: 6,979 (15.38%)
   Points with Δ > 1.0: 2,193 (4.83%)
✓ Saved: ../data/output/figures/maximal_curation_agreement.pdf
✓ Saved: ../data/output/figures/maximal_curation_agreement.html

📊 Statistics for Maximal Curation (Metadata Match):
   Number of pairs: 45,379
   R²: 0.8780
   Spearman ρ: 0.9377
   Median Absolute Error: 0.0000
   Points with Δ > 0.3: 6,979 (15.38%)
   Points with Δ > 1.0: 2,193 (4.83%)


## Figure 8: Overlap Filtering Comparison - Min vs Max Curation

Side-by-side comparison of the two curation levels showing the impact on data quality.

In [20]:
# Create comparison bar chart showing key metrics for Min vs Max curation
if 'min' in overlap_data and 'max' in overlap_data:
    # Find pchembl columns
    min_df = overlap_data['min']
    max_df = overlap_data['max']
    
    # Try to find pchembl columns
    def find_pchembl_cols(df):
        cols = [c for c in df.columns if 'pchembl' in c.lower()]
        if len(cols) >= 2:
            return cols[0], cols[1]
        return None, None
    
    min_col1, min_col2 = find_pchembl_cols(min_df)
    max_col1, max_col2 = find_pchembl_cols(max_df)
    
    if min_col1 and max_col1:
        # Calculate metrics for both
        min_delts = np.abs(min_df[min_col1] - min_df[min_col2])
        max_delts = np.abs(max_df[max_col1] - max_df[max_col2])
        
        min_r2 = r2_score(min_df[min_col1], min_df[min_col2])
        max_r2 = r2_score(max_df[max_col1], max_df[max_col2])
        
        min_spearman = stats.spearmanr(min_df[min_col1], min_df[min_col2])[0]
        max_spearman = stats.spearmanr(max_df[max_col1], max_df[max_col2])[0]
        
        min_mae = median_absolute_error(min_df[min_col1], min_df[min_col2])
        max_mae = median_absolute_error(max_df[max_col1], max_df[max_col2])
        
        min_frac_03 = (min_delts > 0.3).sum() / len(min_delts)
        max_frac_03 = (max_delts > 0.3).sum() / len(max_delts)
        
        min_frac_10 = (min_delts > 1.0).sum() / len(min_delts)
        max_frac_10 = (max_delts > 1.0).sum() / len(max_delts)
        
        # Create comparison figure
        fig_compare = make_subplots(
            rows=1, cols=2,
            subplot_titles=('Data Size & Agreement', 'Error Rates'),
            horizontal_spacing=0.15,
        )
        
        # Left: Size and R² comparison
        categories1 = ['Pairs', 'Unique Molecules']
        min_vals1 = [len(min_df), min_df['InchiKey'].nunique() if 'InchiKey' in min_df.columns else len(min_df)]
        max_vals1 = [len(max_df), max_df['InchiKey'].nunique() if 'InchiKey' in max_df.columns else len(max_df)]
        
        fig_compare.add_trace(
            go.Bar(name='Minimal Curation', x=categories1, y=min_vals1,
                   marker_color='#3498DB', text=[f'{v:,}' for v in min_vals1],
                   textposition='outside', textfont=dict(size=14)),
            row=1, col=1
        )
        fig_compare.add_trace(
            go.Bar(name='Maximal Curation', x=categories1, y=max_vals1,
                   marker_color='#E74C3C', text=[f'{v:,}' for v in max_vals1],
                   textposition='outside', textfont=dict(size=14)),
            row=1, col=1
        )
        
        # Right: Error rates
        categories2 = ['Δ > 0.3', 'Δ > 1.0', 'MAE']
        min_vals2 = [min_frac_03 * 100, min_frac_10 * 100, min_mae * 10]  # Scale MAE for visibility
        max_vals2 = [max_frac_03 * 100, max_frac_10 * 100, max_mae * 10]
        
        fig_compare.add_trace(
            go.Bar(name='Minimal Curation', x=categories2, y=min_vals2,
                   marker_color='#3498DB', text=[f'{v:.1f}%' if i < 2 else f'{v/10:.2f}' for i, v in enumerate(min_vals2)],
                   textposition='outside', textfont=dict(size=14), showlegend=False),
            row=1, col=2
        )
        fig_compare.add_trace(
            go.Bar(name='Maximal Curation', x=categories2, y=max_vals2,
                   marker_color='#E74C3C', text=[f'{v:.1f}%' if i < 2 else f'{v/10:.2f}' for i, v in enumerate(max_vals2)],
                   textposition='outside', textfont=dict(size=14), showlegend=False),
            row=1, col=2
        )
        
        fig_compare.update_layout(
            title=dict(
                text='Overlap Filtering: Minimal vs Maximal Curation Comparison',
                font=dict(size=22, family='Arial'),
                x=0.5, xanchor='center',
            ),
            barmode='group',
            plot_bgcolor='white',
            paper_bgcolor='white',
            width=1200,
            height=500,
            legend=dict(x=0.5, y=1.12, xanchor='center', orientation='h'),
            margin=dict(t=120),
        )
        
        fig_compare.update_yaxes(
            title_text='Count', title_font=dict(size=16),
            showgrid=True, gridcolor='rgba(200,200,200,0.3)',
            row=1, col=1
        )
        fig_compare.update_yaxes(
            title_text='Percentage / Scaled MAE', title_font=dict(size=16),
            showgrid=True, gridcolor='rgba(200,200,200,0.3)',
            row=1, col=2
        )
        
        # Save using centralized helper (includes SVG)
        out_base = output_dir / "overlap_curation_comparison"
        save_plotly_figure(fig_compare, out_base, scale=3)
        
        fig_compare.show()
        
        # Print comparison table
        print("\n" + "=" * 70)
        print("OVERLAP CURATION COMPARISON")
        print("=" * 70)
        print(f"{'Metric':<25} {'Minimal':<20} {'Maximal':<20}")
        print("-" * 70)
        print(f"{'Overlapping Pairs':<25} {len(min_df):>15,} {len(max_df):>15,}")
        print(f"{'Unique Molecules':<25} {min_vals1[1]:>15,} {max_vals1[1]:>15,}")
        print(f"{'R²':<25} {min_r2:>15.4f} {max_r2:>15.4f}")
        print(f"{'Spearman ρ':<25} {min_spearman:>15.4f} {max_spearman:>15.4f}")
        print(f"{'MAE':<25} {min_mae:>15.4f} {max_mae:>15.4f}")
        print(f"{'% with Δ > 0.3':<25} {min_frac_03*100:>14.1f}% {max_frac_03*100:>14.1f}%")
        print(f"{'% with Δ > 1.0':<25} {min_frac_10*100:>14.1f}% {max_frac_10*100:>14.1f}%")
        print("=" * 70)
else:
    print("⚠ Need both min and max overlap data for comparison")

✓ Saved: ../data/output/figures/overlap_curation_comparison.svg
✓ Saved: ../data/output/figures/overlap_curation_comparison.png
✓ Saved: ../data/output/figures/overlap_curation_comparison.png
✓ Saved: ../data/output/figures/overlap_curation_comparison.pdf
✓ Saved: ../data/output/figures/overlap_curation_comparison.html
✓ Saved: ../data/output/figures/overlap_curation_comparison.pdf
✓ Saved: ../data/output/figures/overlap_curation_comparison.html



OVERLAP CURATION COMPARISON
Metric                    Minimal              Maximal             
----------------------------------------------------------------------
Overlapping Pairs                  63,589          45,379
Unique Molecules                   14,076          11,099
R²                                 0.8016          0.8780
Spearman ρ                         0.8969          0.9377
MAE                                0.0000          0.0000
% with Δ > 0.3                      22.9%           15.4%
% with Δ > 1.0                       7.5%            4.8%
